In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
from rtree import index
import time
import re
import folium
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings("ignore")

In [2]:
class SpatialDatabase:
    def __init__(self, file_path):
        self.file_path = file_path
        self.gdf = self.read_csv_and_convert_to_geodataframe(file_path)
        self.spatial_index = self.build_spatial_index()

    def build_spatial_index(self):
        idx = index.Index()
        for i, geom in enumerate(self.gdf.geometry):
            idx.insert(i, geom.bounds)
        return idx

    def read_csv_and_convert_to_geodataframe(self, file_path):
        df = pd.read_csv(file_path, encoding='utf-8')
        geometry = [Point(xy) for xy in zip(df['wgs_lng'], df['wgs_lat'])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
        return gdf

    def nearest_neighbor_query(self, query_point, type_regex_str):
        # Filter points by type_code
        filtered_points = self.gdf[self.gdf['type_code'].astype(str).str.match(type_regex_str)]

        if filtered_points.empty:
            return "No matching POI found with the specified type_code."

        # Find nearest point among filtered points
        nearest_idx = filtered_points.geometry.distance(query_point).idxmin()
        nearest_poi = self.gdf.loc[[nearest_idx]].copy()
        nearest_poi['type_code'] = nearest_poi['type_code'].astype(str)

        return nearest_poi

    def range_query(self, query_range, type_regex_str):
        min_lng, min_lat, max_lng, max_lat = query_range  # Unpack the query range
        query_area = box(min_lng, min_lat, max_lng, max_lat)  # Construct the query area
        possible_matches_index = list(self.spatial_index.intersection(query_area.bounds))
        possible_matches = self.gdf.iloc[possible_matches_index].copy()
        possible_matches['type_code'] = possible_matches['type_code'].astype(str)
        result = possible_matches[possible_matches['type_code'].str.contains(type_regex_str)]
        return result


    def brute_force_range_query(self, query_range, type_regex_str):
        query_area = box(*query_range)
        result = self.gdf[
            (self.gdf['wgs_lng'] >= query_area.bounds[0]) &
            (self.gdf['wgs_lng'] <= query_area.bounds[2]) &
            (self.gdf['wgs_lat'] >= query_area.bounds[1]) &
            (self.gdf['wgs_lat'] <= query_area.bounds[3])
        ].copy()

        result['type_code'] = result['type_code'].astype(str)
        result = result[result['type_code'].str.contains(type_regex_str)]
        return result

    def brute_force_nearest_neighbor_query(self, query_point, type_regex_str):
        nearest_poi = self.gdf.iloc[self.gdf.geometry.distance(query_point).idxmin()]
        if nearest_poi['type_code'].startswith(type_regex_str):
            return nearest_poi
        else:
            return None

In [3]:
def plot_map(initial_point, found_points):
    # Create a map centered at the initial point
    map_center = [initial_point.y, initial_point.x]
    m = folium.Map(location=map_center, zoom_start=15)

    # Add marker for initial point
    folium.Marker(
        location=[initial_point.y, initial_point.x],
        popup='Initial POI',
        icon=folium.Icon(color='blue')
    ).add_to(m)

    # Add markers for found ATMs
    marker_cluster = MarkerCluster().add_to(m)
    for idx, row in found_points.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=f"Type Code: {row['type_code']}",
            icon=folium.Icon(color='green')
        ).add_to(marker_cluster)

    return m

In [4]:
file_path = 'Assignment2-2012_BIT_POI.csv'
data = SpatialDatabase(file_path)

In [5]:
def measure_execution_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

In [6]:
query_point = Point(116.310, 39.955)
radius = 1000  # in meters
query_area = query_point.buffer(radius / 111000)  # Convert meters to degrees (approximately)
type_regex_str = "^5"
result_index, time_index = measure_execution_time(data.range_query, query_area.bounds, type_regex_str)
result_scan, time_scan = measure_execution_time(data.brute_force_range_query, query_area.bounds, type_regex_str)

print("Execution time with spatial index:", time_index)
print("Execution time without spatial index (Brute-force):", time_scan)

Execution time with spatial index: 0.004369020462036133
Execution time without spatial index (Brute-force): 0.001964569091796875


In [7]:
df = pd.read_csv('Assignment2-2012_BIT_POI.csv')
df.head()

,name,wgs_lat,wgs_lng,type_code,base_type,sub_type,category
0,寰太大厦员工餐厅,39.958029,116.317624,50100,餐饮服务,中餐厅,中餐厅
1,一手店（北三环西路辅路）,39.965707,116.313841,50000,餐饮服务,餐饮相关场所,餐饮相关
2,三博冒菜馆,39.966749,116.317380,50100,餐饮服务,中餐厅,中餐厅
3,奇豆の恋,39.967321,116.318733,50700,餐饮服务,冷饮店,冷饮店
4,茶真乡,39.953802,116.316822,50400,餐饮服务,休闲餐饮场所,休闲餐饮场所


In [8]:
query_point = Point(116.311, 39.958)
type_regex_str = "^1603"
width = 0.01
height = 0.01

# Calculate the minimum and maximum longitude and latitude values
min_lng = query_point.x - (width / 2)
max_lng = query_point.x + (width / 2)
min_lat = query_point.y - (height / 2)
max_lat = query_point.y + (height / 2)

# Create the query_range tuple
query_range = (min_lng, min_lat, max_lng, max_lat)

# Pass the individual values of query_range to the box() function
query_area = box(query_range[0], query_range[1], query_range[2], query_range[3])

# Now you can use query_area in your range_query method
range_atm = data.range_query(query_area.bounds, type_regex_str)
range_atm

,name,wgs_lat,wgs_lng,type_code,base_type,sub_type,category,geometry
1157,中国工商银行ＡＴＭ（理工大学储蓄所）,39.959395,116.309544,160302,金融保险服务,自动提款机,中国工商银行ATM,POINT (116.30954 39.95939)
1091,交通银行ＡＴＭ（放光社区西）,39.962941,116.308857,160305,金融保险服务,自动提款机,交通银行ATM,POINT (116.30886 39.96294)
1136,中国工商银行２４小时自助银行（魏公村储蓄所）,39.953313,116.312723,160302,金融保险服务,自动提款机,中国工商银行ATM,POINT (116.31272 39.95331)
1158,交通银行ＡＴＭ（韦伯豪家园北）,39.953381,116.313634,160305,金融保险服务,自动提款机,交通银行ATM,POINT (116.31363 39.95338)
1172,中国银行ＡＴＭ,39.953308,116.309591,160301,金融保险服务,自动提款机,中国银行ATM,POINT (116.30959 39.95331)
1160,北京银行ＡＴＭ（魏公村南区北）,39.953308,116.309591,160316,金融保险服务,自动提款机,北京银行ATM,POINT (116.30959 39.95331)
1159,中国光大银行ＡＴＭ（北京外国语大学社区北）,39.954074,116.306761,160310,金融保险服务,自动提款机,中国光大银行ATM,POINT (116.30676 39.95407)
1170,中国工商银行ＡＴＭ（北京外国语大学社区北）,39.954157,116.306760,160302,金融保险服务,自动提款机,中国工商银行ATM,POINT (116.30676 39.95416)
1120,招商银行ＡＴＭ（魏公村路８号院东北）,39.956512,116.310542,160306,金融保险服务,自动提款机,招商银行ATM,POINT (116.31054 39.95651)


In [9]:
nearest_atm = data.nearest_neighbor_query(query_point, type_regex_str)

print("Nearest ATM (type_code starting with '1603XX'):")
print(nearest_atm)

Nearest ATM (type_code starting with '1603XX'):
                    name    wgs_lat     wgs_lng type_code base_type sub_type  \
1120  招商银行ＡＴＭ（魏公村路８号院东北）  39.956512  116.310542    160306    金融保险服务    自动提款机   

     category                    geometry  
1120  招商银行ATM  POINT (116.31054 39.95651)  


In [10]:
map_with_points = plot_map(query_point, nearest_atm)
map_with_points

In [11]:
type_regex_str = "^1702"
range_office = data.range_query(query_area.bounds, type_regex_str)
range_office.head()

,name,wgs_lat,wgs_lng,type_code,base_type,sub_type,category,geometry
534,北京海涛艺术装饰设计中心,39.961997,116.310869,170201,公司企业,公司,广告装饰,POINT (116.31087 39.96200)
558,德丰杰龙脉创业投资咨询有限公司,39.959594,116.315862,170200,公司企业,公司,公司,POINT (116.31586 39.95959)
851,北京广视通达网络技术有限公司（海淀科技大厦）,39.959594,116.315862,170206,公司企业,公司,网络科技,POINT (116.31586 39.95959)
541,北京科技风险投资股份有限公司,39.959594,116.315862,170200,公司企业,公司,公司,POINT (116.31586 39.95959)
433,中海纪元数字技术发展有限公司（海淀科技大厦）,39.959594,116.315862,170206,公司企业,公司,网络科技,POINT (116.31586 39.95959)


In [12]:
nearest_office = data.nearest_neighbor_query(query_point, type_regex_str)

print("Nearest office (type_code starting with '1203XX'):")
print(nearest_office)

Nearest office (type_code starting with '1203XX'):
             name    wgs_lat     wgs_lng type_code base_type sub_type  \
534  北京海涛艺术装饰设计中心  39.961997  116.310869    170201      公司企业       公司   

    category                    geometry  
534     广告装饰  POINT (116.31087 39.96200)  


In [13]:
map_with_points = plot_map(query_point, nearest_office)
map_with_points

In [14]:
query_point = Point(116.310, 39.955)

buffer_radius = 500 
query_area = query_point.buffer(buffer_radius / 111000)  # Convert meters to degrees (approximately)

type_regex_str = "^5"

result = data.range_query(query_area.bounds, type_regex_str)

num_restaurants = result[result['type_code'].str.startswith('5')].shape[0]

print("Number of restaurants (type_code starting with '5XXXX') within 500 meters:")
print(num_restaurants)

Number of restaurants (type_code starting with '5XXXX') within 500 meters:
40


In [15]:
map_with_restaurants = plot_map(query_point, result)
map_with_restaurants